# Introdução ao Theano

Theano é uma biblioteca de computação simbólica, que possui várias funcionalidades que facilitam o treinamento de redes neurais (e outros modelos de Aprendizagem de Máquina): implementação de várias funcões de álgebra linear; execução em CPU ou GPU de forma transparente e, principamente, diferenciação automática (descrita abaixo)


In [ ]:
import theano
from theano import tensor as T    # Normalmente o módulo tensor é importado com o alias T
theano.config.floatX = 'float64'

## Variaveis simbolicas

Nessa biblioteca, cálculos são expressos como um grafo de computação. Por exemplo, vamos criar duas varíaveis escalares: a e b:


In [ ]:
a = T.scalar()
b = T.scalar()

Vamos definir c como a soma de a e b. O resultado é uma outra variável simbólica:

In [ ]:
c = a + b
type(c)

Isto é, ao somar duas variáveis, não estamos imediatamente calculando o valor da soma, e sim construindo um grafo que descreve a função:

![](images/simple_graph.svg)

Para executar esse cálculo, precisamos primeiro compilar uma função. A sintaxe é a seguinte:

```
f = theano.function([lista_de_entradas], saida)
```

onde o primeiro argumento é uma lista de entradas da função (variáveis que são usadas para computar o resultado), e o segundo argumento é a variável (ou lista de variáveis) de saída.

In [ ]:
soma = theano.function([a,b],c)

In [ ]:
soma(2,3)

Uma forma alternativa para obter o resultado de uma expressão é usar a função eval, que imediatamente compila a função e a chama com os argumentos fornecidos (útil para Debug):

```
resultado = saida.eval({entrada1: valor, entrada2: valor})
```

onde **saida** é a variável simbólica que queremos computar, **entrada1/2** são as variáveis simbólicas de entrada, e **valor** são os valores assinalados para a variável

In [ ]:
c.eval({a: 2, b: 3})

## Variáveis compartilhadas (Shared Variables)

Variáveis compartilhadas do Theano se comportam como variáveis simbólicas (podendo ser usadas em expressões simbólicas), e também como variáveis normais do Python (possuindo um "estado" permanente). 

Esse tipo de variável é muito útil para modelos de aprendizagem de máquina, onde queremos variáveis que mantenham um estado entre chamadas diferentes (por exemplo, os pesos de um modelo).

Para criarmos uma variável compartilhada, usamos a seguinte função:

```
variavel = theano.shared(valor_inicial)
```

Por exemplo, considere um modelo linear, que computa a saída $y(x) = \textbf{w}^\intercal \textbf{x}$, isto é, o produto entre os vetores w e x. 

Vamos iniciar a variável $\textbf{w}$ com um vetor de duas posições: $[1, 0.5]$

In [ ]:
import numpy as np 
w_inicial = np.array([1., 0.5])

w = theano.shared(w_inicial)
print type(w)

Podemos ver que $w$ é uma variável simbólica, mas podemos obter seu valor usando a função:

```
variavel.get_value()
```

De forma similar, podemos atualizar seu valor utilizando:

```
variavel.set_value(novo_valor)
```

In [ ]:
print('valor anterior: %s' % w.get_value())

w.set_value([2, 0.5])
print ('Novo valor: %s' % w.get_value())

O exemplo acima mostra que a variável compartilhada pode ser usada como uma variável normal em Python (mantendo um estado).

O exemplo abaixo mostra como podemos usá-la em uma expressão simbólica:

In [ ]:
x = T.vector()
y = w.dot(x)    # a função dot implementa produto interno de vetores (e também matrizes)

f = theano.function([x], y) # Note que w não é passado na função, pois é uma variável compartilhada

In [ ]:
print(f([5,6]))

Onde o resultado usa o valor atual da variável $w$: $  5 \times 2 + 6 \times 0.5 = 13$.

Note que para compilarmos a função f, informamos apenas $x$ como entrada. Isto é, não precisamos informar as variáveis compartilhadas durante a compilação/chamada da função, visto que a variável mantém um estado.

### Exercício

Declare três variáveis:
* a (variável compartilhada, com valor 3)
* b (variável compartilhada, com valor 1)
* x (variável simbólica)

Implemente uma função que calcule $y = ax + b$. Compile a função para calcular esse valor, e execute com o valor de $x = 5$.

Nota: nesse exercício, todas as variáveis são escalares (use T.scalar() para a variável $x$, theano.shared para $a$ e $b$)

Nota 2: Esse [Link](http://www.deeplearning.net/software/theano/library/tensor/basic.html) contém a descrição das funcionalidades básicas do Theano

In [ ]:
#Sua solução:


In [ ]:
%load solutions/theano_shared.py

## Diferenciação automática

A grande vantagem de usar variáveis simbólicas é que isso permite diferenciação automática. 
Em vários problemas de Machine Learning, precisamos calcular a derivada parcial de uma função de custo, com relação a cada um dos parâmetros do modelo. Ao definir a função de custo como um grafo de operações, Theano pode então usar a regra da cadeia para automaticamente calcular tais derivadas.

### Exemplo prático

Consideremos a seguinte equação:

$$c = a^2 + 3b$$

Podemos usar o método T.grad para calcular as derivadas parciais $\frac{\delta c}{\delta a}$ e $\frac{\delta c}{\delta b}$

A sintaxe do comando é:

```
dy_dx = T.grad(y, x)
```

Onde **y** é uma variável simbólica que representa a expressão a ser diferenciada, e **x** é a entrada. O resultado é outra variável simbólica, que contém a expressão para calcular $\frac{\delta y}{\delta x}$

In [ ]:
a = T.scalar()
b = T.scalar()

c = T.pow(a,2) + 3 * b  # c = a^2 + 3b

In [ ]:
dc_da = T.grad(c, a)
dc_db = T.grad(c, b)

Podemos agora criar uma função que retorna o valor das derivadas para dados valores de **a** e **b**

In [ ]:
g = theano.function([a,b], [dc_da, dc_db])

In [ ]:
print g(5,6)

Sem diferenciação automática, precisaríamos calcular manualmente as derivadas parciais:

$$\frac{\delta c}{\delta a} = 2a $$
$$\frac{\delta c}{\delta b} = 3 $$

In [ ]:
print('dc/da = %g' % (2 * 5))
print('dc/db = %g' % 3)

# Exercício:

Implemente a seguinte função em Theano:

$$ y = \log (2x) $$

Use diferenciação automática para calcular

$$ y'(10) $$

Que indica a derivada de y respectivo à x, na posição $x = 10$

Nota: Utilize a função T.log (a versão simbólica de np.log)

In [ ]:
#Sua reposta:


In [ ]:
%load solutions/theano_diff.py


# Aplicação: Regressão logística

Nesse exercício, vamos implementar regressão logística para um problema em duas dimensões.

Considere uma base de dados $\mathcal{D} = \{\textbf{x}^{(i)}, y^{(i)}\}_{i=1}^N$, composto de N exemplos $(\textbf{x},y)$, onde $\textbf{x}$ é um vetor de entrada de 2 dimensões, e $y$ é a classe $y \in \{0, 1\}$. 

O modelo de regressão logística é um modelo linear, onde a probabilidade do exemplo $x$ pertencer à classe $y = 1$ é dado por:

$$ \hat{y} = P(y=1 | x) = \sigma(\textbf{w}^\intercal\textbf{x} + b)$$

Onde $\sigma(x) = \frac{1}{1 + e^{-x}}$ é a função logística.

O objetivo da regressão logística é otimizar os parâmetros $\textbf{w}$ e $b$ de forma a maximizar a probabilidade da classe correta em cada exemplo da base de treinamento. Para tanto, devemos minimizar a seguinte função de custo:


$$L = -\frac{1}{N} \sum_{i=1}^N{y^{(i)} \log\hat{y}^{(i)}  + (1-y^{(i)}) \log(1 - \hat{y}^{(i)}) }$$


Nesse exercício, vamos otimizar os parametros $\textbf{w}$ e $b$ usando descida de gradiente (Gradient Descent). Nesse método, calculamos a derivada parcial da função de custo com respeito aos parâmetros do modelo, e iterativamente os atualizamos, de acordo com a seguinte regra:

$$\textbf{w} = \textbf{w} - \alpha * \frac{\delta L}{\delta \textbf{w}}$$

Onde $\alpha$ é o tamanho do passo (Learning Rate), e $\frac{\delta L}{\delta \textbf{w}}$ é a derivada parcial da função de custo com respeito à $\textbf{w}$

Nesse exemplo, vamos usar uma base de dados sintética, que é criada abaixo:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
np.random.seed(12)

#Nesse exemplo, geramos duas classes de acordo com distribuições gaussianas, com parametros diferentes
c1 = np.random.multivariate_normal(mean=(2,3), cov=[[2,-1],[-1,2]], size=(1000))
c2 = np.random.multivariate_normal(mean=(5,6), cov=[[2,-1],[-1,1.5]], size=(1000))

#concatenando em uma matrix X e vetor y
data_x = np.concatenate((c1,c2))
data_y = np.concatenate((np.ones(len(c1)), np.zeros(len(c2))))

In [ ]:
#Funções auxiliares para plotar gráficos dos dados e da função de decisão:
def plot_data(c1,c2):
    f,ax=plt.subplots(figsize=(6,5))
    
    ax.scatter(c1[:,0], c1[:,1], c = 'red', marker='+', s=20)
    ax.scatter(c2[:,0], c2[:,1], c = 'white', s=20, edgecolor='b')
    
    ax.set_xlabel('lightness', fontsize=15)
    ax.set_ylabel('width', fontsize=15)
    ax.set_xlim([-1,9])
    ax.set_ylim([-2,12])
    return f,ax

def plot_decision_boundary(c1,c2, w,b):
    gety = lambda x: -(w[0] * x + b) / w[1]
    f, ax = plot_data(c1,c2)
    ax.plot([-2,10], [gety(-2),gety(10)])

In [ ]:
plot_data(c1,c2);

Primeiramente, vamos dividir a base de dados em treinamento/validação e teste

In [ ]:
N = data_x.shape[0]

indices = np.arange(N) # Cria uma sequencia 1,2,...,N
np.random.shuffle(indices) # Permuta aleatóriamente a sequencia

# Vamos separar em 80% para treinamento, 10% para validação, 10% para teste
last_train_indice = int(0.8 * N)
last_val_indice = int(0.9 * N)

train_indices = indices[0:last_train_indice]
val_indices = indices[last_train_indice: last_val_indice]
test_indices = indices[last_val_indice:]

x_train, y_train = data_x[train_indices], data_y[train_indices]
x_val, y_val = data_x[val_indices], data_y[val_indices]
x_test, y_test = data_x[test_indices], data_y[test_indices]

### Definição do modelo

O modelo logístico retorna predições usando a seguinte equação:

$$ \hat{y} =  \sigma(\textbf{w}^\intercal\textbf{x} + b)$$

Vamos primeiro definir os valores iniciais para $\textbf{w}$ e $b$:
$$\textbf{w} \sim U(-0.001, 0.001) $$

$$b = 0$$

Onde U(a,b) retorna números aleatórios segundo uma distribuição uniforme entre $[a,b]$

In [ ]:
w_init = np.random.uniform(-0.001,0.001, size=(2))
b_init = 0.

### Exercício

Nesse exercício, o objetivo é construir uma expressão simbólica que compute $\hat{y}$. Vamos implementar esse cálculo para todos os exemplos da base de dados ao mesmo tempo, usando operações de matrizes. 

Nesse caso, consideramos como entrada uma matrix $X$, onde cada linha da matriz possui um exemplo (portanto, ela possui tamanho Nx2, onde N é o número de exemplos).

Usando essa notação, podemos calcular $\hat{y}$ para todos os vetores de entrada usando a seguinte expressão:
$$\textbf{z} = X\textbf{w} + b$$
$$ \hat{\textbf{y}} =  \sigma(\textbf{z})$$


onde 
$$\sigma(z) = \frac{1}{1 + e^{-z}}$$
Para tanto, é necessário:
 * Definir as variáveis simbólicas $x$ (matrix), $w$ (vetor) e $b$ (escalar)
     * Note que $\textbf{w}$ e $b$ são variáveis compartilhadas. Use theano.shared, com w_init e b_init como inicializacao
 * Implementar as expressões acima para computar $\textbf{z}$ e $\hat{\textbf{y}}$ 
 
Nota:
 * Para implementar a função sigmoid, utilize T.exp (a versão simbólica equivalente à np.exp)

In [ ]:
#Sua solução:

x = 
w = 
b = 

z =
y_hat = 

In [ ]:
#Execute essa célula para ver a resposta

%load solutions/theano_yhat.py

### Exercício

O próximo passo é implementar a função de custo:

$$L = -\frac{1}{N} \sum_{i=1}^N{y^{(i)} \log\hat{y}^{(i)}  + (1-y^{(i)}) \log(1 - \hat{y}^{(i)}) }$$

Para tanto, é preciso:

* Definir a variável simbólica $y$ (um vetor simbolico, que irá conter a classe correta de cada exemplo de treinamento)
* Implementar a função de custo


Dica: As operações aritméticas (soma, multiplicação, logaritmo), se aplicadas em um vetor, são executadas individualmente para cada elemento do vetor. É possível implementar a função acima sem nenhum loop:

$$\textbf{L}_\text{vetor} = -\big(\textbf{y} \log\hat{\textbf{y}}  + (1-\textbf{y}) \log(1 - \hat{\textbf{y}})\big)$$

$$ L = \text{mean}(\textbf{L}_\text{vetor})$$

```
loss = (codigo para calcular o custo para cada elemento)
loss = loss.mean() 
```

Nota: utilize T.log, a versão simbólica de np.log para calcular o logaritmo

In [ ]:
#Sua solução:

y = 
loss = 

In [ ]:
#Verificando a função de custo: para valores aleatórios, y_hat será 0.5 na média, 
#e portanto a função de custo deveria ser próxima a -log(0.5) ~=0.69

loss.eval({x:np.random.random((100,2)), y:np.zeros(100)})

In [ ]:
#Execute essa célula para ver a resposta

%load solutions/theano_loss.py

### Exercício

Para treinarmos o modelo, precisamos calcular a derivada parcial da função de custo referente à cada parametro do modelo. 

Use a função T.grad para calcular a derivada da variável ```loss``` relativo às variáveis $\textbf{w}$ e $b$

In [ ]:
#Sua solução:

w_grad = 
b_grad = 

In [ ]:
#Execute essa célula para ver a resposta

%load solutions/theano_grad.py

Vamos também calcular a taxa de acerto do modelo:

Consideramos que o modelo prevê $y=1$ se a saída $\hat{y} > 0.5$ e $y=0$ caso contrário

In [ ]:
prediction = T.ge(y_hat, 0.5)
accuracy = T.eq(prediction, y).mean()

Por fim, vamos compilar a função que fará o treinamento.

Seguindo o algoritmo de descida de gradiente, faremos a atualização dos parâmetros de acordo com a regra:
$$ w = w - \alpha \nabla_w L$$

onde $\alpha$ é o tamanho do passo (também conhecido como Learning Rate).

Ao compilarmos uma função do Theano, podemos passar uma lista de expressões a serem **atualizadas**, da seguinte forma:

```
updates = [
   (variavel, novo_valor),
   ...
]
```

Dessa forma, a cada chamada da função, os valores de saída serão retornados **e** as variáveis serão atualizadas

In [ ]:
alpha = theano.shared(0.1)

updates = [
    (w, w - alpha  * w_grad),
    (b, b - alpha * b_grad)
]

train_fn = theano.function([x, y], [loss, accuracy], updates = updates)
valid_fn = theano.function([x, y], [loss, accuracy])

Dessa forma, com a função de treinamento compilada, basta iterativamente chamá-la com os valores de $X$ e $y$ da base de treinamento, e os valores de $\textbf{w}$ e $b$ serão automaticamente atualizados a cada chamada

## Visualizando a fronteira de decisão com os parametros iniciais

In [ ]:
plot_decision_boundary(c1,c2, w.get_value(),b.get_value())

In [ ]:
cost_history = []
acc_history = []
val_cost_history = []
val_acc_history = []

### Executando o treinamento por 50 iterações:

In [ ]:
for i in range(50):
    cost, acc = train_fn(x_train,y_train) # w e b são atualizados aqui
    cost_history.append(cost)
    acc_history.append(acc)
    
    val_cost, val_acc = valid_fn(x_val, y_val) # Essa chamada não muda os parametros
    val_cost_history.append(val_cost)
    val_acc_history.append(val_acc)

In [ ]:
plot_decision_boundary(c1,c2, w.get_value(), b.get_value())

### Executando o treinamento por mais 3000 iterações:

In [ ]:
for i in range(3000):
    cost, acc = train_fn(x_train,y_train)
    cost_history.append(cost)
    acc_history.append(acc)
    
    val_cost, val_acc = valid_fn(x_val, y_val)
    val_cost_history.append(val_cost)
    val_acc_history.append(val_acc)

In [ ]:
plot_decision_boundary(c1,c2, w.get_value(), b.get_value())

In [ ]:
plt.plot(cost_history, 'b--', label='Treinamento')
plt.plot(val_cost_history, 'r-', label='Validacao')
plt.xlabel('Numero de iteracoes', fontsize=15)
plt.ylabel('Custo', fontsize=15)
plt.legend()

In [ ]:
plt.plot(acc_history, 'b--', label='Treinamento')
plt.plot(val_acc_history, 'r-', label='Validacao')
plt.xlabel('Numero de iteracoes', fontsize=15)
plt.ylabel('Taxa de acerto', fontsize=15)
plt.legend(loc=4)

In [ ]:
get_preds = theano.function([x], y_hat)

In [ ]:
test_acc = np.mean((get_preds(x_test) > 0.5) == y_test)

In [ ]:
print 'Acerto na base de teste: ', test_acc

# Exercício - critério de parada

No exercício acima, executamos o treinamento para um número fixo de iterações. Nesse exercício vamos implementar um outro critério de parada, baseado na convergência.

O critério de parada será baseado no progresso de cada iteração. Considere a diminuição de erro entre duas iterações sucessivas:

$$ \Delta L^{(t)} = L^{(t-1)} - L^{(t)} $$

Onde $L^{(t)}$ indica o resultado da função de custo (do treinamento) na iteração $t$.

Vamos parar o algoritmo na iteração $t$ que satisfaça:

$$ \Delta L^{(t)} < \epsilon $$ 

O pseudo-código da função de treinamento é:

* funcao Treino(maximo_iteracoes, epsilon)
   * para $t$ de 1 a maximo_iteracoes:
       * $L^{(t)}$ = train_fn(x_train, y_train)
       * $L_\text{val}^{(t)}$ = valid_fn(x_val, y_val)
       * $\Delta t = L^{(t-1)} - L^{(t)}$
       * se $\Delta t < epsilon$
            * Fim do loop
            
Implemente a função de treino acima. Ao fim do treinamento, imprima o número de iterações executadas


Nota: cuidado para não fazer a verificação na primeira iteração (pois $L^{(t-1)}$ é indefinido)

In [ ]:
#Vamos re-iniciar os pesos do modelo, para os valores aleatórios iniciais:
def reset_weights():
    w.set_value(w_init)
    b.set_value(b_init)
reset_weights()


In [ ]:
# Sua solução

def train(train_fn, valid_fn, x_train, y_train, x_val, y_val, max_iterations, epsilon):
    cost_history = []  # historico de custo. cost_history[t] será o custo na iteracao t (L(t) no algoritmo acima)
    val_cost_history = []
    # Escreva aqui o codigo. Retorne uma lista do custo em treinamento e validação, 
    # como fizemos anteriormente usando "cost_history" e "val_cost_history"

In [ ]:
#Execute essa célula para ver a solucao
%load solutions/theano_parada.py

In [ ]:
# Vamos testar a função de treino com epsilon = 1e-5 = 0.00001

reset_weights()
cost_history, val_cost_history = train(train_fn, valid_fn, x_train, y_train, x_val, y_val, 
      max_iterations = 10000, 
      epsilon = 1e-5)

print 'Funcao de custo em treino: %.4f' % (cost_history[-1])

In [ ]:
plot_decision_boundary(c1,c2, w.get_value(), b.get_value())

Execute a função acima usando valores diferentes de epsilon (e.g. 1e-3, 1e-4) e note o numero de iterações para convergencia.

## Exercício extra (avançado)

No exemplo acima, utilizamos o algoritmo de descida de gradiente para fazer a otimização.
Alguns modelos (como redes neurais pequenas, ou regressão logística) podem ser otimizados de forma mais eficiente utilizando algoritmos de segunda ordem, como L-BFGS.

Implemente a otimização do modelo de regressão logística usando o método scipy.optimize.fmin_bfgs ([manual](http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_bfgs.html))

Nota: para isso, você precisará definir uma função "f":

```
def f(params):
    (...)
    return custo, gradiente
```

* Argumento: um vetor com todos os parâmetros do modelo (valores de $w$ e $b$, com todos os valores em um único vetor)
* Retorno: 
 * "custo" (um escalar contendo o valor da função de custo para os valores de $w$ e $b$)
 * "gradiente" (um vetor contendo a derivada parcial da função de custo referente à cada parâmetro do modelo)
 
Por exemplo, considere que $w$ contenha 2 valores, e $b$ contenha um valor. O vetor params conterá 3 valores; custo será um escalar, e o vetor gradiente conterá 3 valores.

* Como você pode obter o valor de $w$ e $b$ a partir do vetor "params"? E para juntar os valores de  $\frac{d_L}{d_w}$ e $\frac{d_L}{d_b}$ no vetor "gradiente"?? 
* Como fazer para alterar as variáveis compartilhadas usando os valores informados de $w$ e $b$?